In [ ]:
from dataclasses import dataclass
from typing import Protocol

from hypernodes import Pipeline, node

# Data Entities

In [ ]:
@dataclass
class Document:
    text: str
    score: float
    metadata: dict = None


@dataclass
class Answer:
    text: str
    sources: list[Document]

# Protocols

In [ ]:
class VectorDB(Protocol):
    """Interface for vector database implementations."""

    def search(self, query: str, k: int) -> list[Document]:
        """Search for k most similar documents."""
        ...


class LLM(Protocol):
    """Interface for language model implementations."""

    def generate(self, prompt: str) -> str:
        """Generate text from prompt."""
        ...


# Implementations

In [ ]:
class SimpleVectorDB:
    """Simple in-memory vector database using sentence-transformers."""

    def __init__(self, documents: list[str], model_name: str = "all-MiniLM-L6-v2"):
        from sentence_transformers import SentenceTransformer

        self.model = SentenceTransformer(model_name)
        self.documents = documents
        self.embeddings = self.model.encode(documents)

    def search(self, query: str, k: int) -> list[Document]:
        """Cosine similarity search."""
        import numpy as np

        query_embedding = self.model.encode([query])[0]

        # Compute cosine similarity
        similarities = np.dot(self.embeddings, query_embedding) / (
            np.linalg.norm(self.embeddings, axis=1) * np.linalg.norm(query_embedding)
        )

        # Get top-k indices
        top_k_indices = np.argsort(similarities)[-k:][::-1]

        return [
            Document(
                text=self.documents[idx],
                score=float(similarities[idx]),
                metadata={"index": int(idx)},
            )
            for idx in top_k_indices
        ]


class OpenAILLM:
    """OpenAI-based LLM."""

    def __init__(self, model: str = "gpt-4o-mini", temperature: float = 0.7):
        from openai import OpenAI

        self.client = OpenAI()
        self.model = model
        self.temperature = temperature

    def generate(self, prompt: str) -> str:
        """Generate response using OpenAI API."""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
        )
        return response.choices[0].message.content

In [ ]:
class MockLLM:
    """OpenAI-based LLM."""

    def __init__(self):
        pass

    def generate(self, prompt: str) -> str:
        return f"This is a mock response to {prompt}"

# Pipeline Nodes

In [ ]:
@node(output_name="retrieved_docs")
def retrieve(query: str, vector_db: VectorDB, top_k: int = 5) -> list[Document]:
    """Retrieve most relevant documents from vector database."""
    return vector_db.search(query, k=top_k)


@node(output_name="answer")
def generate(query: str, retrieved_docs: list[Document], llm: LLM) -> Answer:
    """Generate answer using LLM with retrieved context."""
    context = "\n\n".join([f"[{i}] {doc.text}" for i, doc in enumerate(retrieved_docs)])
    prompt = f"Context:\n{context}\n\nQuestion: {query}"

    llm_response = llm.generate(prompt)

    return Answer(text=llm_response, sources=retrieved_docs)

# Build Pipeline

In [ ]:
rag_pipeline = Pipeline(nodes=[retrieve, generate])
rag_pipeline.visualize()

# Example Usage

In [ ]:
# Sample corpus
corpus = [
    "Photosynthesis is the process by which plants convert light energy into chemical energy.",
    "The mitochondria is the powerhouse of the cell.",
    "DNA contains genetic instructions for living organisms.",
    "Water is composed of hydrogen and oxygen atoms.",
]

In [ ]:
# Initialize components
vector_db = SimpleVectorDB(documents=corpus)
# llm = OpenAILLM(model="gpt-4o-mini", temperature=0.3)

In [ ]:
llm = MockLLM()

## `.run` with one values

In [ ]:
inputs = {
    "query": "How do plants create energy?",
    "vector_db": vector_db,
    "llm": llm,
    "top_k": 2,
}
# Run pipeline
result = rag_pipeline.run(
    inputs=inputs,
)

In [ ]:
print(f"Answer: {result['answer'].text}")
print("\nSources:")
for doc in result["answer"].sources:
    print(f"  - {doc.text} (score: {doc.score:.3f})")

Answer: This is a mock response to Context:
[0] Photosynthesis is the process by which plants convert light energy into chemical energy.

[1] The mitochondria is the powerhouse of the cell.

Question: How do plants create energy?

Sources:
  - Photosynthesis is the process by which plants convert light energy into chemical energy. (score: 0.676)
  - The mitochondria is the powerhouse of the cell. (score: 0.358)


## Add Callbacks

In [ ]:
from hypernodes.telemetry import ProgressCallback

rag_pipeline.with_callbacks([ProgressCallback()])

# Run pipeline
results = rag_pipeline.run(
    inputs=inputs,
)

pipeline_4561326384   0%|          | 0/2 [00:00<?, ?it/s]

retrieve   0%|          | 0/1 [00:00<?, ?it/s]

generate   0%|          | 0/1 [00:00<?, ?it/s]

## Add Cache

In [ ]:
from hypernodes.cache import DiskCache

cache = DiskCache(path="cache")
rag_pipeline.with_cache(cache)

# Run pipeline
results = rag_pipeline.run(
    inputs=inputs,
)  # Runs regularly

pipeline_4561326384   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Run pipeline again
results = rag_pipeline.run(
    inputs=inputs,
)  # Cached


pipeline_4561326384   0%|          | 0/2 [00:00<?, ?it/s]

## `.map` over multiple values

In [ ]:
# Run pipeline
multi_inputs = {
    "query": ["How do plants create energy?", "What is the capital of France?"],
    "vector_db": vector_db,
    "llm": llm,
    "top_k": 2,
}
results = rag_pipeline.map(
    inputs=multi_inputs,
    map_over="query",
)

Running pipeline_4561326384 with 2 examples...   0%|          | 0/2 [00:00<?, ?it/s]

retrieve   0%|          | 0/2 [00:00<?, ?it/s]

generate   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Print results
for result in results:
    print(f"Answer: {result['answer'].text}")

Answer: This is a mock response to Context:
[0] Photosynthesis is the process by which plants convert light energy into chemical energy.

[1] The mitochondria is the powerhouse of the cell.

Question: How do plants create energy?
Answer: This is a mock response to Context:
[0] The mitochondria is the powerhouse of the cell.

[1] Water is composed of hydrogen and oxygen atoms.

Question: What is the capital of France?


# Nested Pipelines - Evaluation Example

Now let's use the RAG pipeline as a component in a larger evaluation pipeline.


In [ ]:
@dataclass
class EvaluationResult:
    query: str
    generated_answer: str
    ground_truth: str
    score: float
    match: bool

In [ ]:
@node(output_name="evaluation")
def evaluate_answer(answer: Answer, ground_truth: str, query: str) -> EvaluationResult:
    """Simple evaluation: check if key terms from ground truth appear in answer."""
    # Simple keyword-based evaluation (could use LLM-based eval in production)
    ground_truth_lower = ground_truth.lower()
    answer_lower = answer.text.lower()

    # Extract key terms (simple approach)
    key_terms = [word for word in ground_truth_lower.split() if len(word) > 4]
    matches = sum(1 for term in key_terms if term in answer_lower)

    score = matches / len(key_terms) if key_terms else 0.0

    return EvaluationResult(
        query=query,
        generated_answer=answer.text,
        ground_truth=ground_truth,
        score=score,
        match=score > 0.5,
    )


In [ ]:
# Use RAG pipeline as a node in evaluation pipeline
eval_pipeline = Pipeline(nodes=[rag_pipeline.as_node(name="RAG"), evaluate_answer])

In [ ]:
# Visualize the nested structure
eval_pipeline.visualize()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# Visualize the nested structure with depth=2
eval_pipeline.visualize(depth=2)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Run Evaluation on Single Example


In [ ]:
eval_inputs = {
    "query": "How do plants create energy?",
    "ground_truth": "Plants create energy through photosynthesis by converting light into chemical energy.",
    "vector_db": vector_db,
    "llm": llm,
    "top_k": 2,
}

result = eval_pipeline.run(inputs=eval_inputs)


In [ ]:
print(f"Query: {result['evaluation'].query}")
print(f"Generated: {result['evaluation'].generated_answer}")
print(f"Ground Truth: {result['evaluation'].ground_truth}")
print(f"Score: {result['evaluation'].score:.2f}")
print(f"Match: {result['evaluation'].match}")


Query: How do plants create energy?
Generated: This is a mock response to Context:
[0] Photosynthesis is the process by which plants convert light energy into chemical energy.

[1] The mitochondria is the powerhouse of the cell.

Question: How do plants create energy?
Ground Truth: Plants create energy through photosynthesis by converting light into chemical energy.
Score: 0.78
Match: True


## Batch Evaluation with `.map()`

Now let's evaluate multiple query-ground_truth pairs at once.


In [ ]:
test_cases = [
    {
        "query": "How do plants create energy?",
        "ground_truth": "Plants create energy through photosynthesis by converting light into chemical energy.",
    },
    {
        "query": "What is DNA?",
        "ground_truth": "DNA contains genetic instructions for living organisms.",
    },
    {
        "query": "What is water made of?",
        "ground_truth": "Water is composed of hydrogen and oxygen atoms.",
    },
]

batch_inputs = {
    "query": [tc["query"] for tc in test_cases],
    "ground_truth": [tc["ground_truth"] for tc in test_cases],
    "vector_db": vector_db,
    "llm": llm,
    "top_k": 2,
}

results = eval_pipeline.map(inputs=batch_inputs, map_over=["query", "ground_truth"])

In [ ]:
for i, result in enumerate(results):
    eval_result = result["evaluation"]
    print(f"\n=== Test Case {i + 1} ===")
    print(f"Query: {eval_result.query}")
    print(f"Score: {eval_result.score:.2f}")
    print(f"Match: {'✓' if eval_result.match else 'X'}")

# Overall accuracy
accuracy = sum(r["evaluation"].score for r in results) / len(results)
print(f"\nOverall Accuracy: {accuracy:.2%}")



=== Test Case 1 ===
Query: How do plants create energy?
Score: 0.78
Match: ✓

=== Test Case 2 ===
Query: What is DNA?
Score: 1.00
Match: ✓

=== Test Case 3 ===
Query: What is water made of?
Score: 1.00
Match: ✓

Overall Accuracy: 92.59%


# Use Daft Engine



In [ ]:
from hypernodes.engines import DaftEngine

engine = DaftEngine()

eval_pipeline.with_engine(engine)

results = eval_pipeline.map(inputs=batch_inputs, map_over=["query", "ground_truth"])

🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 UDF retrieve-6ce28a38-5902-4c5c-bfe0-5b8021c0572c: 00:00 

🗡️ 🐟 UDF generate-ed41c824-35de-4a98-b1e6-a9a75b43acbe: 00:00 

🗡️ 🐟 UDF batch_udf-47e78bd4-5dc3-497e-80b1-13e04fb9e647: 00:00 

In [ ]:
results[0]

{'answer': Answer(text='This is a mock response to Context:\n[0] Photosynthesis is the process by which plants convert light energy into chemical energy.\n\n[1] The mitochondria is the powerhouse of the cell.\n\nQuestion: How do plants create energy?', sources=[Document(text='Photosynthesis is the process by which plants convert light energy into chemical energy.', score=0.6764461398124695, metadata={'index': 0}), Document(text='The mitochondria is the powerhouse of the cell.', score=0.35782390832901, metadata={'index': 1})]),
 'retrieved_docs': [Document(text='Photosynthesis is the process by which plants convert light energy into chemical energy.', score=0.6764461398124695, metadata={'index': 0}),
  Document(text='The mitochondria is the powerhouse of the cell.', score=0.35782390832901, metadata={'index': 1})],
 'evaluation': EvaluationResult(query='How do plants create energy?', generated_answer='This is a mock response to Context:\n[0] Photosynthesis is the process by which plants 

# Async

In [ ]:
import asyncio

from hypernodes import stateful


@stateful
class AsyncClient:
    def __init__(self, base_url: str):
        self.base_url = base_url


@node(output_name="response")
async def fetch(item_id: str, client: AsyncClient) -> str:
    await asyncio.sleep(0.001)
    return f"{client.base_url}/{item_id}"


client = AsyncClient(base_url="http://api.com")
pipeline = Pipeline(nodes=[fetch])
result = pipeline.run(inputs={"item_id": "test", "client": client})

In [ ]:
result

{'response': 'http://api.com/test'}

In [35]:
@node(output_name="hello")
def a(x: int) -> int:
    return x + 1


@node
def b(x: int, y: int) -> int:
    return x + y


pipeline = Pipeline(nodes=[a, b])
pipeline.visualize()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
pipeline.run(inputs={"x": 1}, output_name="a")

ValueError: Missing required input(s): ['y']. Required inputs: ['x', 'y']